<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [347]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [312]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from __future__ import division
from __future__ import print_function
import os,sys
import gzip 
import numpy 
import rpy2 
import rpy2.robjects as robjects
import pyplinkseq

## Assessing independent effects of protein-altering *NOD2* alleles 

In this section we jointly analyze the genotype and phenotype data across the eight associated enriched alleles.

In [313]:
## Set PLINK/SEQ project
pyplinkseq.set_project('ibdxjup')
pyplinkseq.locdbattach('/humgen/gsa-hphome1/rivas/ibdseq/resources/locdb')
pyplinkseq.seqdbattach('/humgen/gsa-hphome1/rivas/ibdseq/resources/seqdb')
pyplinkseq.annotate_load('gencode')

In [314]:
inds = pyplinkseq.ind_fetch('indiv=@vcftmp/ajcd.indivs', 'cd')
var = pyplinkseq.var_fetch('indiv=@vcftmp/ajcd.indivs downcode reg=chr16:50756540,chr16:50763778,chr16:50750810,chr16:50733392,chr16:50745656,chr16:50745926,chr16:50744892,chr16:50744565')

The list of attributes available for instance <code>inds</code> and array of instances <code>var</code>: 

In [315]:
inds.__dict__.keys()

['PHENO', 'ID']

In [316]:
phenotypedict = {}
phenotypearr = []
for i in range(0,len(inds.PHENO.PHENOTYPE)):
    phenotypearr.append(int(inds.PHENO.PHENOTYPE[i]) - 1)
    phenotypedict[inds.ID[i]] = int(inds.PHENO.PHENOTYPE[i]) - 1
phenotypearr = numpy.array(phenotypearr)

In [317]:
var[0].__dict__.keys()

['BP1', 'CHR', 'BP2', 'NS', 'ID', 'CON']

The <code>Variant</code> instance supports <code>BP1, CHR, BP2, NS, ID, CON</code>, which rerepresents the beginning genomic coordinate, the chromosome, end of the spanning genomic coordinate, number of samples, variant id, and a <code>SampleVariant</code> instance: <code>CON</code>. 

In [318]:
var[0].CON

<pyplinkseq.SampleVariant instance at 0x7fdf512a5e18>

In [319]:
var[0].CON.__dict__.keys()

['QUAL', 'FSET', 'REF', 'ALT', 'GENO']

In [320]:
var[0].CON.GENO

<pyplinkseq.Genotype instance at 0x7fdf4f4a1758>

The <code>SampleVariant</code> instance supports <code>QUAL, FSET, REF, ALT, GENO</code>, which rerepresents the quality of the variant, FSET, reference allele, alternate allele, and a <code>Genotype</code> instance: <code>GENO</code>, which contains the genotype data.

In [321]:
var[0].CON.GENO.__dict__.keys()

['GT']

In [322]:
varids = []
for i in range(0,len(var)):
    varids.append(str(var[i].CHR) + ':' + str(var[i].BP1) + '_' + str(var[i].CON.REF) + '/' + str(var[i].CON.ALT))

The variants we are analyzing correspond to the following alleles in *NOD2*: 

In [323]:
for i in range(0,len(var)):
    details = pyplinkseq.annotate(var[i].CHR,var[i].BP1,var[i].CON.REF,var[i].CON.ALT,"details","")
    hgvs = [j.split('=')[1] for j in details.split(',') if j.split('=')[0] == 'HGVS']
    prachange = pyplinkseq.annotate(var[i].CHR,var[i].BP1,var[i].CON.REF,var[i].CON.ALT,"protein","")
    if len(hgvs) > 1:
        print(str(var[i].CHR) + ':' + str(var[i].BP1) + '_' + var[i].CON.REF + '/' + var[i].CON.ALT,hgvs[0])
    else:
        print(str(var[i].CHR) + ':' + str(var[i].BP1) + '_' + var[i].CON.REF + '/' + var[i].CON.ALT,prachange)

16:50733392_T/A .
16:50744565_T/G .,p.248L>R
16:50744892_A/C p.357D>A
16:50745656_G/A,T p.612A>T,p.612A>S
16:50745926_C/T p.702R>W
16:50750810_A/G p.852N>S
16:50756540_G/C,T p.48G>R,p.48G>C,p.908G>R,p.908G>C
16:50763778_G/GC p.L147PfsX1


Let's create a multi-dimensional array of genotypes.

In [324]:
genotypelists = []
for varnum in xrange(int(var.__len__())):
    genos = []
    for genotypenum in xrange(len(var[varnum].CON.GENO.GT)):
        genos.append(var[varnum].CON.GENO.GT[genotypenum])
    genos = [gt if gt >= 0 else 0 for gt in genos]
    genotypelists.append(genos)

In [325]:
# Indices for R702W, fs1007insC, and G908R: 4,6,7 ; N852S: 5 ; add A612T : 3 ; add splice region variant : 0 ; add L248R : 1 ; add D357A : 2
genotypelists = numpy.array(genotypelists)
genotypelists = genotypelists.T
print(genotypelists.shape,phenotypearr.shape)

(4912, 8) (4912,)


We will use Bayesian model averaging package from Adrian Raftery https://cran.r-project.org/web/packages/BMA/BMA.pdf  

In [326]:
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

x = genotypelists
nr,nc = x.shape
xr = ro.r.matrix(x, nrow=nr, ncol=nc)
ro.r.assign("xr", x)
ro.r.assign("var.ids", robjects.StrVector(varids))
y = phenotypearr
nr = y.shape[0]
nc = 1
yr = ro.r.matrix(y, nrow=nr, ncol=nc)
ro.r.assign("yr", y)

robjects.r('''
library("BMA")
rafnod2 <- function(x,y){
colnames(x) <- var.ids
glm.out.FF <- bic.glm(x, y, strict = FALSE, OR = 30,
glm.family="binomial", factor.type=FALSE)
summary(glm.out.FF)
print(glm.out.FF$probne0)
png("bicglm.png")
imageplot.bma(glm.out.FF)
dev.off()

pdf("posterior_bmaseFINALv2.pdf")
par(mar=c(7.1, 4.1, 4.1, 2.1))
plot(0,0,col="white", xlim=c(0,1+length(var.ids)),ylim=c(-.5,6),xlab="",xaxt="n",ylab="Posterior odds ratios",main="",xaxs="i",yaxs="i",cex.lab=1.2,cex.axis=1.3,cex.main=1.3)
abline(h=0,lty=2)
for(i in 1:length(var.ids)){
    arrows(i,exp(glm.out.FF$postmean[i+1] - glm.out.FF$postsd[i+1]),i,exp(glm.out.FF$postmean[i+1] + glm.out.FF$postsd[i+1]), code = 0, angle = 90, lwd=1.5, col = rgb(0,0,0))
    points(i,exp(glm.out.FF$postmean[i+1]),pch=16,cex=1.5, col = rgb(0,0,0))
 }
axis(1,at=seq(1,length(var.ids),1),labels=var.ids, las = 2, cex.axis = .7)
dev.off()

png("posterior_mle.png")
par(mar=c(7.1, 4.1, 4.1, 2.1))
plot(0,0,col="white", xlim=c(0,1+length(var.ids)),ylim=c(-.5,6),xlab="",xaxt="n",ylab="Best model MLE odds ratios",main="",xaxs="i",yaxs="i",cex.lab=1.2,cex.axis=1.3,cex.main=1.3)
abline(h=0,lty=2)
for(i in 1:length(var.ids)){
arrows(i,exp(glm.out.FF$mle[1,i+1] - glm.out.FF$se[1,i+1]),i,exp(glm.out.FF$mle[1,i+1] + glm.out.FF$se[1,i+1]), code = 0, angle = 90, lwd=1.5, col = rgb(0,0,0,.5))
  points(i,exp(glm.out.FF$mle[1,i+1]),pch=16,cex=1.5, col = rgb(0,0,0,.5))
  }
axis(1,at=seq(1,length(var.ids),1),labels=var.ids, las = 2, cex.axis = .7)
dev.off()
print(glm.out.FF$postmean)
print(glm.out.FF$postsd)
return(glm.out.FF)

}
''')
pltf = robjects.globalenv['rafnod2']
pltf = robjects.r['rafnod2']
res = pltf(xr,yr)





Call:
bic.glm.matrix(x = x, y = y, glm.family = "binomial", strict = FALSE,     OR = 30, factor.type = FALSE)


  3  models were selected
 Best  3  models (cumulative posterior probability =  1 ): 

                    p!=0    EV      SD       model 1     model 2     model 3   
Intercept           100    -0.7863  0.03652  -7.851e-01  -7.934e-01  -7.551e-01
X16.50733392_T.A     96.5   0.4711  0.15405   4.862e-01   4.939e-01       .    
X16.50744565_T.G    100.0   0.9721  0.25164   9.738e-01   9.701e-01   9.544e-01
X16.50744892_A.C     27.3   0.2190  0.39317       .       8.030e-01       .    
X16.50745656_G.A.T  100.0   1.4138  0.26994   1.415e+00   1.411e+00   1.416e+00
X16.50745926_C.T    100.0   0.5407  0.12960   5.390e-01   5.468e-01   5.275e-01
X16.50750810_A.G    100.0   0.7239  0.16211   7.236e-01   7.266e-01   7.076e-01
X16.50756540_G.C.T  100.0   0.8521  0.09474   8.520e-01   8.536e-01   8.415e-01
X16.50763778_G.GC   100.0   1.0078  0.10736   1.009e+00   1.005e+00   9.996e-01


<code>p!=0</code> represents the posterior probability that each variable is non-zero (in percent).

![title](img/bicglm.png)

The dominating models include all the variants and possibly exclude D357A from the calculations. 

![Posterior odds ratio (after model average)](img/posterior_bmaseFINALv2.png)

We show the the posterior standard odds ratio and standard error (from model averaging), and below the posterior maximum likelihood odds ratio and standard error for the model with the highest probability (model 1).

![title](img/posterior_mle.png)

## Assessing additivity in *NOD2*

In this section we evaluate whether any departure from additivity exists for the NOD2 alleles. Hence, we evaluate RR if 0,1, or 2 alleles. 

In [327]:
nod2dosage = numpy.sum(x,axis = 1)
nod2dosage[nod2dosage > 2] = 2 
nod2dosage.shape

(4912,)

In [328]:
xnod2d = nod2dosage
nr= xnod2d.shape[0]
nc = 1
xrnod2d = ro.r.matrix(xnod2d, nrow=nr, ncol=nc)
ro.r.assign("xr", xrnod2d)
yd = phenotypearr
nr = yd.shape[0]
nc = 1
yrd = ro.r.matrix(yd, nrow=nr, ncol=nc)
ro.r.assign("yr", yd)

robjects.r('''
genotypenod2 <- function(x,y){
 # genotype
 fit.gen <- glm(y ~ as.factor(x), family = "binomial")
 # additive  
 fit.add <- glm(y ~ as.numeric(x), family = "binomial")
 print("GENOTYPE")
 print(summary(fit.gen))
 print("ADDITIVE")
 print(summary(fit.add))
 gen.phen <- data.frame(cbind(y,x))
 colnames(gen.phen) <- c("pheno","nod2dosage")
 print(table(gen.phen))
}
''')
pltf = robjects.globalenv['genotypenod2']
pltf = robjects.r['genotypenod2']
res = pltf(xrnod2d,yrd)


[1] "GENOTYPE"

Call:
glm(formula = y ~ as.factor(x), family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7773  -0.8757  -0.8757   1.2485   1.5128  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -0.76087    0.03654  -20.82   <2e-16 ***
as.factor(x)1  0.59535    0.06851    8.69   <2e-16 ***
as.factor(x)2  2.10942    0.15704   13.43   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 6514.3  on 4911  degrees of freedom
Residual deviance: 6239.2  on 4909  degrees of freedom
AIC: 6245.2

Number of Fisher Scoring iterations: 4

[1] "ADDITIVE"

Call:
glm(formula = y ~ as.numeric(x), family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5380  -0.8629  -0.8629   1.1730   1.5287  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -0.79619    

Let's exclude the frameshift indel from consideration:

In [329]:
nod2dosagesansfs = numpy.sum(x[:,0:7],axis = 1)
nod2dosagesansfs[nod2dosagesansfs > 2] = 2 
nod2dosagesansfs.shape

xd = nod2dosagesansfs
nr= xd.shape[0]
nc = 1
xrd = ro.r.matrix(xd, nrow=nr, ncol=nc)
ro.r.assign("xr", xrd)
yd = phenotypearr
nr = yd.shape[0]
nc = 1
yrd = ro.r.matrix(yd, nrow=nr, ncol=nc)
ro.r.assign("yr", yd)

robjects.r('''
genotypenod2 <- function(x,y){
 # genotype
 fit.gen <- glm(y ~ as.factor(x), family = "binomial")
 # additive  
 fit.add <- glm(y ~ as.numeric(x), family = "binomial")
 print("GENOTYPE")
 print(summary(fit.gen))
 print("ADDITIVE")
 print(summary(fit.add))
 gen.phen <- data.frame(cbind(y,x))
 colnames(gen.phen) <- c("pheno","nod2dosage")
 print(table(gen.phen))
 print(anova(fit.add,fit.gen))
 print(pchisq(11.135,2,lower.tail = F))
}
''')
pltf = robjects.globalenv['genotypenod2']
pltf = robjects.r['genotypenod2']
res = pltf(xrd,yrd)

[1] "GENOTYPE"

Call:
glm(formula = y ~ as.factor(x), family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7622  -0.9011  -0.9011   1.4816   1.4816  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -0.69153    0.03483 -19.853   <2e-16 ***
as.factor(x)1  0.60909    0.07046   8.644   <2e-16 ***
as.factor(x)2  2.00636    0.21203   9.463   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 6514.3  on 4911  degrees of freedom
Residual deviance: 6341.1  on 4909  degrees of freedom
AIC: 6347.1

Number of Fisher Scoring iterations: 4

[1] "ADDITIVE"

Call:
glm(formula = y ~ as.numeric(x), family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5136  -0.8946  -0.8946   1.4895   1.4895  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -0.70926    

Let's try the multi-locus model:

In [330]:
## Compound het analysis 
idx = numpy.unique(numpy.where(x == 2)[0])
idx.sort()
len(idx)
x2 = numpy.delete(x,idx,0)
x2.shape


(4854, 8)

In [332]:
## exclude fs indel
nod2dosagesansfs = numpy.sum(x2[:,0:7],axis = 1)
nod2dosagesansfs[nod2dosagesansfs > 2] = 2 
nod2dosagesansfs.shape

(4854,)

In [333]:
xd = nod2dosagesansfs
nr= xd.shape[0]
nc = 1
xrd = ro.r.matrix(xd, nrow=nr, ncol=nc)
ro.r.assign("xr", xrd)
yd = numpy.delete(phenotypearr,idx,0)
nr = yd.shape[0]
nc = 1
yrd = ro.r.matrix(yd, nrow=nr, ncol=nc)
ro.r.assign("yr", yd)

robjects.r('''
genotypenod2 <- function(x,y){
 # genotype
 fit.gen <- glm(y ~ as.factor(x), family = "binomial")
 # additive  
 fit.add <- glm(y ~ as.numeric(x), family = "binomial")
 print("GENOTYPE")
 print(summary(fit.gen))
 print("ADDITIVE")
 print(summary(fit.add))
 gen.phen <- data.frame(cbind(y,x))
 colnames(gen.phen) <- c("pheno","nod2dosage")
 print(table(gen.phen))
  print(anova(fit.add,fit.gen))
 print(pchisq(8.9986,2,lower.tail = F))
}
''')
pltf = robjects.globalenv['genotypenod2']
pltf = robjects.r['genotypenod2']
res = pltf(xrd,yrd)


[1] "GENOTYPE"

Call:
glm(formula = y ~ as.factor(x), family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7723  -0.8949  -0.8949   1.4891   1.4891  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -0.70825    0.03503 -20.216   <2e-16 ***
as.factor(x)1  0.62580    0.07056   8.869   <2e-16 ***
as.factor(x)2  2.04575    0.24768   8.260   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 6409.8  on 4853  degrees of freedom
Residual deviance: 6255.8  on 4851  degrees of freedom
AIC: 6261.8

Number of Fisher Scoring iterations: 4

[1] "ADDITIVE"

Call:
glm(formula = y ~ as.numeric(x), family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5007  -0.8896  -0.8896   1.4956   1.4956  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -0.72271    

How about if we make sure that we are not simply measuring R702W and G908R together:

In [334]:
## Compound het analysis 
idx = numpy.where(((x[:,4] == 1) & (x[:,6] == 1)) | ((x[:,4] == 1) & (x[:,7] == 1)) | ((x[:,7] == 1) & (x[:,6] == 1)))
idx2 = numpy.where(x == 2) 
idxf = numpy.unique(numpy.append(idx[0],idx2[0], axis = 0))
idxf.sort()




In [335]:
x2 = numpy.delete(x,idxf,0)
x2.shape

(4763, 8)

In [336]:
nod2dosagesansmajor = numpy.sum(x2,axis = 1)
nod2dosagesansmajor[nod2dosagesansmajor > 2] = 2 
nod2dosagesansmajor.shape
xd = nod2dosagesansmajor
nr= xd.shape[0]
nc = 1
xrd = ro.r.matrix(xd, nrow=nr, ncol=nc)
ro.r.assign("xr", xrd)
yd = numpy.delete(phenotypearr,idxf,0)
nr = yd.shape[0]
nc = 1
yrd = ro.r.matrix(yd, nrow=nr, ncol=nc)
ro.r.assign("yr", yd)

robjects.r('''
genotypenod2 <- function(x,y){
 # genotype
 fit.gen <- glm(y ~ as.factor(x), family = "binomial")
 # additive  
 fit.add <- glm(y ~ as.numeric(x), family = "binomial")
 print("GENOTYPE")
 print(summary(fit.gen))
 print("ADDITIVE")
 print(summary(fit.add))
 gen.phen <- data.frame(cbind(y,x))
 colnames(gen.phen) <- c("pheno","nod2dosage")
 print(table(gen.phen))
  print(anova(fit.add,fit.gen))
 print(pchisq(11.271,2,lower.tail = F))
}
''')
pltf = robjects.globalenv['genotypenod2']
pltf = robjects.r['genotypenod2']
res = pltf(xrd,yrd)


[1] "GENOTYPE"

Call:
glm(formula = y ~ as.factor(x), family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7370  -0.8757  -0.8757   1.2485   1.5128  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -0.76087    0.03654 -20.823   <2e-16 ***
as.factor(x)1  0.59535    0.06851   8.690   <2e-16 ***
as.factor(x)2  2.01933    0.22956   8.796   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 6249.7  on 4762  degrees of freedom
Residual deviance: 6092.0  on 4760  degrees of freedom
AIC: 6098

Number of Fisher Scoring iterations: 4

[1] "ADDITIVE"

Call:
glm(formula = y ~ as.numeric(x), family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.4572  -0.8688  -0.8688   1.2078   1.5213  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -0.77973    0.

## Assessing for deviation from non-additivity in *LRRK2* 

In [337]:
inds = pyplinkseq.ind_fetch('indiv=@vcftmp/ajcd.indivs', 'cd')
var = pyplinkseq.var_fetch('indiv=@vcftmp/ajcd.indivs downcode reg=chr12:40740686,chr12:40734202')


In [338]:
varids = []
for i in range(0,len(var)):
    varids.append(str(var[i].CHR) + ':' + str(var[i].BP1) + '_' + str(var[i].CON.REF) + '/' + str(var[i].CON.ALT))

In [339]:
for i in range(0,len(var)):
    details = pyplinkseq.annotate(var[i].CHR,var[i].BP1,var[i].CON.REF,var[i].CON.ALT,"details","")
    hgvs = [j.split('=')[1] for j in details.split(',') if j.split('=')[0] == 'HGVS']
    prachange = pyplinkseq.annotate(var[i].CHR,var[i].BP1,var[i].CON.REF,var[i].CON.ALT,"protein","")
    if len(hgvs) > 1:
        print(str(var[i].CHR) + ':' + str(var[i].BP1) + '_' + var[i].CON.REF + '/' + var[i].CON.ALT,hgvs[0])
    else:
        print(str(var[i].CHR) + ':' + str(var[i].BP1) + '_' + var[i].CON.REF + '/' + var[i].CON.ALT,prachange)

12:40734202_G/A p.2019G>S
12:40740686_A/G p.2081N>D


In [340]:
genotypelists = []
for varnum in xrange(int(var.__len__())):
    genos = []
    for genotypenum in xrange(len(var[varnum].CON.GENO.GT)):
        genos.append(var[varnum].CON.GENO.GT[genotypenum])
    genos = [gt if gt >= 0 else 0 for gt in genos]
    genotypelists.append(genos)

In [341]:
genotypelists = numpy.array(genotypelists)
genotypelists = genotypelists.T
print(genotypelists.shape,phenotypearr.shape)

(4912, 2) (4912,)


In [342]:
xlrrk2 = genotypelists
nr,nc = xlrrk2.shape
xlrrk2r = ro.r.matrix(xlrrk2, nrow=nr, ncol=nc)
ro.r.assign("xlrrk2r", xlrrk2r)
ro.r.assign("var.ids", robjects.StrVector(varids))
y = phenotypearr
nr = y.shape[0]
nc = 1
yr = ro.r.matrix(y, nrow=nr, ncol=nc)
ro.r.assign("yr", y)


<Array - Python:0x7fdf513fdfc8 / R:0x72f1620>
[       0,        0,        0, ...,        1,        1,        1]

In [343]:
lrrk2dosage = numpy.sum(xlrrk2,axis = 1)
lrrk2dosage[lrrk2dosage > 2] = 2 
lrrk2dosage.shape
xlrrk2d = lrrk2dosage
nr = xlrrk2d.shape[0]
nc = 1
xrlrrk2d = ro.r.matrix(xlrrk2d, nrow=nr, ncol=nc)
ro.r.assign("xr", xrlrrk2d)
yd = phenotypearr
nr = yd.shape[0]
nc = 1
yrd = ro.r.matrix(yd, nrow=nr, ncol=nc)
ro.r.assign("yr", yd)

robjects.r('''
genotypenod2 <- function(x,y){
 # genotype
 fit.gen <- glm(y ~ as.factor(x), family = "binomial")
 # additive  
 fit.add <- glm(y ~ as.numeric(x), family = "binomial")
 print("GENOTYPE")
 print(summary(fit.gen))
 print("ADDITIVE")
 print(summary(fit.add))
 gen.phen <- data.frame(cbind(y,x))
 colnames(gen.phen) <- c("pheno","nod2dosage")
 print(table(gen.phen))
 print(anova(fit.add,fit.gen))
 print(pchisq(1.7454,2,lower.tail = F))
}
''')
pltf = robjects.globalenv['genotypenod2']
pltf = robjects.r['genotypenod2']
res = pltf(xrlrrk2d,yrd)

[1] "GENOTYPE"

Call:
glm(formula = y ~ as.factor(x), family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1284  -0.9502  -0.9502   1.4231   1.4231  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -0.56119    0.03197 -17.556  < 2e-16 ***
as.factor(x)1  0.44486    0.08461   5.258 1.46e-07 ***
as.factor(x)2  0.33805    0.38862   0.870    0.384    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 6514.3  on 4911  degrees of freedom
Residual deviance: 6486.5  on 4909  degrees of freedom
AIC: 6492.5

Number of Fisher Scoring iterations: 4

[1] "ADDITIVE"

Call:
glm(formula = y ~ as.numeric(x), family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2842  -0.9514  -0.9514   1.4217   1.4217  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -0.55796    

## Assessing GxG interaction between *NOD2* and *LRRK2* alleles

In [346]:
robjects.r('''
gg.interaction.lrrk2nod2 <- function(x,x2,y){
 # genotype
 fit.gen <- glm(y ~ as.factor(x)*as.factor(x2), family = "binomial")
 print("GENOTYPE")
 print(summary(fit.gen))
}
''')
pltf = robjects.globalenv['gg.interaction.lrrk2nod2']
pltf = robjects.r['gg.interaction.lrrk2nod2']
res = pltf(xrnod2d,xrlrrk2d,yrd)

[1] "GENOTYPE"

Call:
glm(formula = y ~ as.factor(x) * as.factor(x2), family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9174  -0.8511  -0.8511   1.2649   1.5435  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -0.82894    0.03971 -20.875  < 2e-16 ***
as.factor(x)1                  0.62569    0.07450   8.398  < 2e-16 ***
as.factor(x)2                  2.10869    0.16962  12.432  < 2e-16 ***
as.factor(x2)1                 0.48617    0.10499   4.631 3.65e-06 ***
as.factor(x2)2                 0.47227    0.49440   0.955    0.339    
as.factor(x)1:as.factor(x2)1  -0.21393    0.19497  -1.097    0.273    
as.factor(x)2:as.factor(x2)1  -0.10092    0.45618  -0.221    0.825    
as.factor(x)1:as.factor(x2)2  -0.77984    0.88416  -0.882    0.378    
as.factor(x)2:as.factor(x2)2  10.81405  229.62906   0.047    0.962    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

(